<a href="https://colab.research.google.com/github/Shahriar10k/RAG-with-NSU/blob/main/Text_to_csv_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Text Files to CSV**

First upload the zip folder containing all the annotated texts in the colab  runtime folder. Then set your desired size and make the csv. Then download and get nuts.

In [ ]:
import shutil

#file_dir =  Place the directory inside " " of the zip folders uploaded(preferably in "content/") in colab run time.
file_dir = ""

#target_dir= The directory where we want to unzip. By default inside the content folder.
target_dir ="/content/"

# Format of archive file
archive_format = "zip"

shutil.unpack_archive(file_dir, target_dir, archive_format)


Now we will read each text file and use filename as title.

In [ ]:
import os
import urllib

titles = []
articles = []

print('Reading in all articles.....')

i=0

#fold_dir= Place the folder directory of the unzipped text files. Must add a '/'. Ex- "/content/499Dataset/"
fold_dir="/content/499Dataset/"

#Read for each of the files in the subdirectory....
for filename in os.listdir(fold_dir):

    #Skip non text files
    if not filename[-3:] == "txt":
      continue

    #Open the file
    with open(fold_dir+ filename, "rb") as f:
        # Use the file name as the article title (strip the ".txt")
         titles.append(filename[:-4])

         # Read in the article text.
         articles.append(f.read().decode('latin-1'))

print(' Done.\n')

print(' There are {:,} articles.' .format(len(articles)))

Now we will split the text files content for our passage size. These passage size can be 100,200 or anything and then this will be tokenized once more by BERT tokenizer.  

**Use slice_limit variable to set passage length**


In [ ]:
print('Before splitting, {:,} articles.\n'.format(len(titles)))

#Two lists : titles and articles

passage_titles = []
passages = []

#Enter the slicing limit.Ex -100,200,300
slice_limit = 100
print("Splitting.........")

#For each article and its title....

for i in range(len(titles)):

    title = titles[i]
    article = articles[i]

    #Skip if there is no content or empty
    if len(articles) == 0:
      print("Skipping empty article:", title)
      continue

    #Split the text on white space.
    #By default, this will remove all whitespace, including newline tab and 
    #characters
    words = article.split()
    #print("words: \n")
    #print(words)
    #print("\n")

    #Loop over the words, incrementing by slice_limit.
    for i in range(0, len(words),slice_limit):

      #Select the next 100 words
      #Python slices automatically stop at the end of the array.
      chunk_words = words[i : i + slice_limit]
      #print("Chunk words: \n")
      #print(chunk_words)
      #print("\n")

      #Recombine the words into a passage by joining with whitespace
      chunk = " ".join(chunk_words)

      #Remove any trailing whitespace.
      chunk = chunk.strip()

      #To avoid a possible edge case, skip any empty chunks.
      if len(chunk) == 0:
        continue
      
      #Store the chunk. Every chunk in the article uses the article title
    
      #print(chunk)
      passage_titles.append(title)
      passages.append(chunk)

print(' Done.\n')

chunked_corpus = {'title': passage_titles, 'text' : passages}

print('After splitting, {:,} "passages".'.format(len(chunked_corpus['title'])))

Create the csv file. Download the file from colab runtime folder to your local device

In [ ]:
import pandas as pd

df = pd.DataFrame(chunked_corpus)

#Write it out to disk. Comma(,) or tab(\t) separator can be used 
df.to_csv(r"/content/curated_dataset_"+str(slice_limit)+".csv" ,sep=',', index=False)